### **Understanding Data Preprocessing & Constraints**

#### *Getting dataframe form dataset*

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/dataset_task1.csv") # read the csv 
before_df = df.copy()
df # print df

,name,age,city,temperature_F,income,date
0,Alice,25,New York,95.0,55000.0,2021-06-15
1,Bob,thirty,Los Angeles,102.0,60000.0,15/06/2021
2,Carol,35,new york,89.0,NaN,2021/06/16
3,bob,40,Chicago,78.0,70000.0,06-17-2021
4,Alice,1000,Chicago,84.0,45000.0,NaN
5,Eve,29,Boston,98.0,50000.0,2021-06-19
6,NaN,NaN,Boston,NaN,NaN,2021-06-20
7,Bob,thirty,Los Angeles,102.0,60000.0,15/06/2021


In [3]:
df.dtypes # list current datatypes of columns

name              object
age               object
city              object
temperature_F    float64
income           float64
date              object
dtype: object

#### First, we start work with **name column**, we change all NaN (Not a Number) cells to *Anonymous* name, and capitalize first letter + lowercase others (if present)

In [4]:
df.name = df.name.fillna("Anonymous")
df.name = df.name.str.capitalize()
df.name = df.name.where(df.name.apply(lambda x: not isinstance(x, str) or len(x) <= 30), pd.NA) # Check for max len using "apply" and "where"
df.name

0        Alice
1          Bob
2        Carol
3          Bob
4        Alice
5          Eve
6    Anonymous
7          Bob
Name: name, dtype: object

#### As we have multiple columns in age, which have "word" numbers, we should convert them to number, before change dtype of column. To do it, I used a simple python lib: `word2number`, which would do it. Also I used a lamda expresion to convert any columns, which would be str

In [5]:
from word2number import w2n

In [6]:
df.age = df.age.apply(lambda x: w2n.word_to_num(x) if isinstance(x, str) else x)
df.age

0      25.0
1      30.0
2      35.0
3      40.0
4    1000.0
5      29.0
6       NaN
7      30.0
Name: age, dtype: float64

In [7]:
df.age = df.age.apply(lambda x: 0 if (x < 0 or x > 100) else x) # remove outliers
df.age

0    25.0
1    30.0
2    35.0
3    40.0
4     0.0
5    29.0
6     NaN
7    30.0
Name: age, dtype: float64

In [8]:
mean_age = df.age.mean()
df.age = df.age.fillna(mean_age) # NaN values would be replased to their mean
df.age = df.age.replace(0, mean_age) # replace 0 to mean too
df.age = df.age.astype(int)
df.age

0    25
1    30
2    35
3    40
4    27
5    29
6    27
7    30
Name: age, dtype: int64

### Modifying a city column

In [9]:
df.city = df.city.str.title() # We use a title() here, not capitalize(), because we need to capitalize all letters in cells
df.city = df.city.where(df.name.apply(lambda x: not isinstance(x, str) or len(x) <= 30), pd.NA) # Check for max len using "apply" and "where"
df.city

0       New York
1    Los Angeles
2       New York
3        Chicago
4        Chicago
5         Boston
6         Boston
7    Los Angeles
Name: city, dtype: object

### Modify a temperature_F column

In [10]:
df.temperature_F = df.temperature_F.fillna(df.temperature_F.median()) # NaN would be a median values
df.temperature_F = (df.temperature_F - 32) * 5 / 9 # Convert from F to C
df.rename(columns={'temperature_F': 'temperature_C'}, inplace=True) # Rename column
df.temperature_C

0    35.000000
1    38.888889
2    31.666667
3    25.555556
4    28.888889
5    36.666667
6    35.000000
7    38.888889
Name: temperature_C, dtype: float64

### Convert a NaN values in income

In [11]:
df.income = df.income.fillna(df.income.mean())
df.income

0    55000.000000
1    60000.000000
2    56666.666667
3    70000.000000
4    45000.000000
5    50000.000000
6    56666.666667
7    60000.000000
Name: income, dtype: float64

### Working with date column

In [12]:
df.date

0    2021-06-15
1    15/06/2021
2    2021/06/16
3    06-17-2021
4           NaN
5    2021-06-19
6    2021-06-20
7    15/06/2021
Name: date, dtype: object

#### As we have a huge range of dif format date, it is needed to convert them all, so I applied three different funcoitns to convert them to on format.

In [13]:
df.date = pd.to_datetime(df.date, format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m-%d').fillna(df.date)
df.date = pd.to_datetime(df.date, format='%m-%d-%Y', errors='coerce').dt.strftime('%Y-%m-%d').fillna(df.date)
df.date = pd.to_datetime(df.date, format='%Y/%m/%d', errors='coerce').dt.strftime('%Y-%m-%d').fillna(df.date)
df.date

0    2021-06-15
1    2021-06-15
2    2021-06-16
3    2021-06-17
4           NaN
5    2021-06-19
6    2021-06-20
7    2021-06-15
Name: date, dtype: object

In [14]:
### Converting date + fill NaN values

In [15]:
df.date = pd.to_datetime(df.date) # Convert to date dtype to avoid time date in next line
df.date = df.date.fillna(pd.Timestamp('2021-01-01')).dt.date # Take a constant to **NaN** values
df.date = pd.to_datetime(df.date) # Convert again, because previous line would not save date type
df.date

0   2021-06-15
1   2021-06-15
2   2021-06-16
3   2021-06-17
4   2021-01-01
5   2021-06-19
6   2021-06-20
7   2021-06-15
Name: date, dtype: datetime64[ns]

### Remove duplicates

In [19]:
df = df.drop_duplicates()

### Display float only 2 numbers after comma

In [20]:
pd.options.display.float_format = '{:.2f}'.format

### Results:

In [21]:
before_df

,name,age,city,temperature_F,income,date
0,Alice,25,New York,95.00,55000.00,2021-06-15
1,Bob,thirty,Los Angeles,102.00,60000.00,15/06/2021
2,Carol,35,new york,89.00,NaN,2021/06/16
3,bob,40,Chicago,78.00,70000.00,06-17-2021
4,Alice,1000,Chicago,84.00,45000.00,NaN
5,Eve,29,Boston,98.00,50000.00,2021-06-19
6,NaN,NaN,Boston,NaN,NaN,2021-06-20
7,Bob,thirty,Los Angeles,102.00,60000.00,15/06/2021


In [22]:
df

,name,age,city,temperature_C,income,date
0,Alice,25,New York,35.00,55000.00,2021-06-15
1,Bob,30,Los Angeles,38.89,60000.00,2021-06-15
2,Carol,35,New York,31.67,56666.67,2021-06-16
3,Bob,40,Chicago,25.56,70000.00,2021-06-17
4,Alice,27,Chicago,28.89,45000.00,2021-01-01
5,Eve,29,Boston,36.67,50000.00,2021-06-19
6,Anonymous,27,Boston,35.00,56666.67,2021-06-20
